In [1]:
#importing The Required Libraries
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from random import shuffle
import tensorflow as tf
import pandas as pd
from sklearn.metrics import roc_auc_score
import datetime

In [2]:
tf.set_random_seed(1)

In [3]:
#HyperParameters
epochs = 150
step_size = 8
IMG_SIZE_ALEXNET = 227 # image size
validating_size = 20 # while cross validating, we are evaluating batch by batch
nodes_fc1 = 4096 # no of nodes on fc layer 1
nodes_fc2 = 4096 # no of nodes on fc layer 2
output_locations = 4 # x1,x2,y1,y2

In [4]:
#Separating train and validation sets
X_train = np.load('object_localization.npy')
y = np.load('object_localization_y.npy')

In [5]:
data = X_train

In [6]:
#Separating train and validation sets
train = data[:int(len(data)*0.8)]
cv = data[int(len(data)*0.8):]

In [7]:
# X for train input
X = np.array(train)

In [8]:
Y2 = y[:int(len(y)*0.8)]
Y2.shape

(11200, 4)

In [9]:
# cv_x for train input
cv_x = np.array(cv)

In [10]:
cv_y2 = y[int(len(y)*0.8):]
cv_y2.shape

(2800, 4)

In [11]:
#How many training images are kept as 'steps'
steps = len(train)
print(steps)
remaining = steps % step_size

11200


In [12]:
#Resetting graph
tf.reset_default_graph()

In [13]:
#Defining Placeholders
x = tf.placeholder(tf.float32,shape=[None,IMG_SIZE_ALEXNET,IMG_SIZE_ALEXNET,3])
y_true_2 = tf.placeholder(tf.float32,shape=[None,output_locations])

In [14]:
##CONVOLUTION LAYER 1
#Weights for layer 1
w_1 = tf.Variable(tf.truncated_normal([11,11,3,96], stddev=0.01))
#Bias for layer 1
b_1 = tf.Variable(tf.constant(0.0, shape=[96]))
#Applying convolution
c_1 = tf.nn.conv2d(x, w_1,strides=[1, 4, 4, 1], padding='VALID')
#Adding bias
c_1 = c_1 + b_1
#Applying RELU
c_1 = tf.nn.relu(c_1)
print(c_1)

Tensor("Relu:0", shape=(?, 55, 55, 96), dtype=float32)


In [15]:
##POOLING LAYER1
p_1 = tf.nn.max_pool(c_1, ksize=[1, 3, 3, 1],strides=[1, 2, 2, 1], padding='VALID')
print(p_1)

Tensor("MaxPool:0", shape=(?, 27, 27, 96), dtype=float32)


In [16]:
##CONVOLUTION LAYER 2
#Weights for layer 2
w_2 = tf.Variable(tf.truncated_normal([5,5,96,256], stddev=0.01))
#Bias for layer 2
b_2 = tf.Variable(tf.constant(1.0, shape=[256]))
#Applying convolution
c_2 = tf.nn.conv2d(p_1, w_2,strides=[1, 1, 1, 1], padding='SAME')
#Adding bias
c_2 = c_2 + b_2
#Applying RELU
c_2 = tf.nn.relu(c_2)
print(c_2)

Tensor("Relu_1:0", shape=(?, 27, 27, 256), dtype=float32)


In [17]:
##POOLING LAYER2
p_2 = tf.nn.max_pool(c_2, ksize=[1, 3, 3, 1],strides=[1, 2, 2, 1], padding='VALID')
print(p_2)

Tensor("MaxPool_1:0", shape=(?, 13, 13, 256), dtype=float32)


In [18]:
##CONVOLUTION LAYER 3
#Weights for layer 3
w_3 = tf.Variable(tf.truncated_normal([3, 3, 256, 384], stddev=0.01))
#Bias for layer 3
b_3 = tf.Variable(tf.constant(0.0, shape=[384]))
#Applying convolution
c_3 = tf.nn.conv2d(p_2, w_3,strides=[1, 1, 1, 1], padding='SAME')
#Adding bias
c_3 = c_3 + b_3
#Applying RELU
c_3 = tf.nn.relu(c_3)
print(c_3)

Tensor("Relu_2:0", shape=(?, 13, 13, 384), dtype=float32)


In [19]:
##CONVOLUTION LAYER 4
#Weights for layer 4
w_4 = tf.Variable(tf.truncated_normal([3, 3, 384, 384], stddev=0.01))
#Bias for layer 4
b_4 = tf.Variable(tf.constant(0.0, shape=[384]))
#Applying convolution
c_4 = tf.nn.conv2d(c_3, w_4,strides=[1, 1, 1, 1], padding='SAME')
#Adding bias
c_4 = c_4 + b_4
#Applying RELU
c_4 = tf.nn.relu(c_4)
print(c_4)

Tensor("Relu_3:0", shape=(?, 13, 13, 384), dtype=float32)


In [20]:
##CONVOLUTION LAYER 5
#Weights for layer 5
w_5 = tf.Variable(tf.truncated_normal([3, 3, 384, 256], stddev=0.01))
#Bias for layer 5
b_5 = tf.Variable(tf.constant(0.0, shape=[256]))
#Applying convolution
c_5 = tf.nn.conv2d(c_4, w_5,strides=[1, 1, 1, 1], padding='SAME')
#Adding bias
c_5 = c_5 + b_5
#Applying RELU
c_5 = tf.nn.relu(c_5)
print(c_5)

Tensor("Relu_4:0", shape=(?, 13, 13, 256), dtype=float32)


In [21]:
##POOLING LAYER3
p_3 = tf.nn.max_pool(c_5, ksize=[1, 3, 3, 1],strides=[1, 2, 2, 1], padding='VALID')
print(p_3)

Tensor("MaxPool_2:0", shape=(?, 6, 6, 256), dtype=float32)


In [22]:
#Flattening
flattened = tf.reshape(p_3,[-1,6*6*256])
print(flattened)

Tensor("Reshape:0", shape=(?, 9216), dtype=float32)


In [23]:
##Fully Connected Layer 1
#Getting input nodes in FC layer 1
input_size = int( flattened.get_shape()[1] )
#Weights for FC Layer 1
w1_fc = tf.Variable(tf.truncated_normal([input_size, nodes_fc1], stddev=0.01))
#Bias for FC Layer 1
b1_fc = tf.Variable( tf.constant(1.0, shape=[nodes_fc1] ) )
#Summing Matrix calculations and bias
s_fc1 = tf.matmul(flattened, w1_fc) + b1_fc
#Applying RELU
s_fc1 = tf.nn.relu(s_fc1)

In [24]:
#Dropout Layer 1
hold_prob1 = tf.placeholder(tf.float32)
s_fc1 = tf.nn.dropout(s_fc1,keep_prob=hold_prob1)
print(s_fc1)

Tensor("dropout/mul:0", shape=(?, 4096), dtype=float32)


In [25]:
##Fully Connected Layer 2
#Weights for FC Layer 2
w2_fc = tf.Variable(tf.truncated_normal([nodes_fc1, nodes_fc2], stddev=0.01))
#Bias for FC Layer 2
b2_fc = tf.Variable( tf.constant(1.0, shape=[nodes_fc2] ) )
#Summing Matrix calculations and bias
s_fc2 = tf.matmul(s_fc1, w2_fc) + b2_fc
#Applying RELU
s_fc2 = tf.nn.relu(s_fc2)
print(s_fc2)

Tensor("Relu_6:0", shape=(?, 4096), dtype=float32)


In [26]:
#Dropout Layer 2
hold_prob2 = tf.placeholder(tf.float32)
s_fc2 = tf.nn.dropout(s_fc2,keep_prob=hold_prob2)

In [27]:
##Fully Connected Layer 3 -- REGRESSION HEAD
#Weights for FC Layer 3
w3_fc_2 = tf.Variable(tf.truncated_normal([nodes_fc2,output_locations], stddev=0.01))
#Bias for FC Layer 3b3_fc = tf.Variable( tf.constant(1.0, shape=[output_classes] ) )
b3_fc_2 = tf.Variable( tf.constant(1.0, shape=[output_locations] ) )
#Summing Matrix calculations and bias
y_pred_2 = tf.matmul(s_fc2, w3_fc_2) + b3_fc_2
#Applying RELU
print(y_pred_2)

Tensor("add_7:0", shape=(?, 4), dtype=float32)


In [28]:
#Defining Regression Loss
regression_loss = tf.multiply(tf.reduce_mean(tf.square(y_pred_2 - y_true_2)),1.0)

In [29]:
final_loss = regression_loss

In [30]:
#Defining objective function
train = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(final_loss)

In [31]:
#Initializing weights
init = tf.global_variables_initializer()

In [32]:
#Starting Empty lists to keep results
regression_list = []
#In order to save, creating a tf.train.Saver() object.
saver = tf.train.Saver()

In [33]:
#GPU settings
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.allocator_type = 'BFC'

tf.add_to_collection("regression_head", y_pred_2)

In [34]:
print(datetime.datetime.now())

2019-02-12 07:44:52.495998


In [35]:
def main():
    with tf.Session(config=config) as sess:
        sess.run(init)
        for i in range(epochs):
            total_cost = 0
            for j in range(0,steps-remaining,step_size):
                #Feeding step_size-amount data with 0.5 keeping probabilities on DROPOUT LAYERS
                _,c = sess.run([train,final_loss],feed_dict = {x:X[j:j+step_size], y_true_2:Y2[j:j+step_size],hold_prob1:0.5,hold_prob2:0.5})
                total_cost += c
            print("Epoch #",i+1," ====> ",total_cost)
#Writing for loop to calculate test statistics. GTX 1060 isn't able to calculate all cv data.
            cv_regression_list = []
            for v in range(0,len(cv_x)-int(len(cv_x) % validating_size),validating_size):
                coordinates = sess.run([y_pred_2],feed_dict={x:cv_x[v:v+validating_size],hold_prob1:1.0,hold_prob2:1.0})
                regression_loss = np.mean(pow(cv_y2[v:v+validating_size] - coordinates , 2 ) )
                cv_regression_list.append(regression_loss)
            regression_loss_cv_ = round(np.mean(cv_regression_list),5)
            regression_list.append(regression_loss_cv_)
        print("Training has finished and model is saved")
        saver.save(sess, os.path.join(os.getcwd(),"CNN_final.ckpt"))

if __name__ == "__main__":
    main()

Epoch # 1  ====>  1424094.045578003
Epoch # 2  ====>  620275.1808776855
Epoch # 3  ====>  513350.40882873535
Epoch # 4  ====>  435124.075302124
Epoch # 5  ====>  377055.2024040222
Epoch # 6  ====>  335593.94770622253
Epoch # 7  ====>  303683.09481811523
Epoch # 8  ====>  278515.79860305786
Epoch # 9  ====>  260648.21522521973
Epoch # 10  ====>  242485.3102054596
Epoch # 11  ====>  228798.32899093628
Epoch # 12  ====>  213754.29935455322
Epoch # 13  ====>  199259.44234466553
Epoch # 14  ====>  192453.19703292847
Epoch # 15  ====>  177772.89972877502
Epoch # 16  ====>  170328.9782218933
Epoch # 17  ====>  159164.95582199097
Epoch # 18  ====>  151128.22874641418
Epoch # 19  ====>  142570.7325630188
Epoch # 20  ====>  132267.46536254883
Epoch # 21  ====>  124360.31617546082
Epoch # 22  ====>  118896.71995544434
Epoch # 23  ====>  111421.74781036377
Epoch # 24  ====>  105718.66763973236
Epoch # 25  ====>  100218.8476486206
Epoch # 26  ====>  96794.76226329803
Epoch # 27  ====>  92685.837617

In [36]:
print(datetime.datetime.now())

2019-02-12 10:30:57.470116
